In [2]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [3]:
c

Client Scheduler: tcp://127.0.0.1:42471 Dashboard: http://127.0.0.1:34870/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [4]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import numpy.ma as ma

today=date.today()

#https://pmall.univ-grenoble-alpes.fr/docs/auto_examples_code/windrose/windrose-exemple.html

In [5]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
    

In [6]:
data_dir = '/store/molines/NATL60/NATL60-CJM165-S/1d/'
gridfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_mesh_hgr.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [7]:
e1=dsgrid['e1t'][0]
e2=dsgrid['e2t'][0]


In [8]:
imin = {'AC' : 3088, 'NE' :3838, 'GS' : 994}
imax = {'AC' : 3386, 'NE' :4119, 'GS' : 1295}
jmin = {'AC' : 301, 'NE' :1783, 'GS' : 751}
jmax = {'AC' : 657, 'NE' :2239, 'GS' : 1155}


In [10]:
for month in ['march', 'september']:
#for month in ['september']:
    
    if month == 'march':
        Date=['20130301','20130302','20130303','20130304','20130305','20130306','20130307','20130308','20130309','20130310','20130311','20130312','20130313','20130314','20130315','20130316','20130317','20130318','20130319','20130320','20130321','20130322','20130323','20130324','20130325','20130326','20130327','20130328','20130329','20130330']
    if month == 'september':
        Date=['20120901','20120902','20120903','20120904','20120905','20120906','20120907','20120908','20120909','20120910','20120911','20120912','20120913','20120914','20120915','20120916','20120917','20120918','20120919','20120920','20120921','20120922','20120923','20120924','20120925','20120926','20120927','20120928','20120929','20120930']

    for date in Date:
        print(date)
        
        year=date[0:4]
        month=date[4:6]
        day=date[6:]

        filename = sorted(glob.glob(data_dir+str(year)+'/NATL60-CJM165_y'+str(year)+'m'+str(month)+'d'+str(day)+'.1d_gridTsurf.nc'))
        file=filename[0]
        print(file)

        ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1})
        dataT=ds['sosstsst']
        dataS=ds['sosaline']
        dataT_dx=dx_var(dataT,e1)
        dataS_dx=dx_var(dataS,e1)
        dataT_dy=dy_var(dataT,e1)
        dataS_dy=dy_var(dataS,e1)
        R=(dataT_dx + 1j * dataT_dy)/(dataS_dx + 1j * dataS_dy)
        magR=np.abs(R)
        magR_ma=ma.masked_invalid(magR)
        Tu=np.arctan(magR)
        
#        for box in ['AC','NE','GS']:
        for box in ['NE','GS']:
#        for box in ['AC']:

            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_'+str(date)+'.1d_'+str(box)+'.nc'
            if not os.path.exists(filename): 
                
                print('compute magR for box '+str(box))
                magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
                Tu_box=Tu[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
                magR_box_nonan=magR_box[~np.isnan(magR_box)]
                Tu_box_nonan=Tu_box[~np.isnan(magR_box)]
                magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
                Tu_box_nonan_noinf=Tu_box_nonan[~np.isinf(magR_box_nonan)]
                magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
                Tu_box_nonan_noinf=Tu_box_nonan_noinf.reset_index('z')

                print('write /'+filename)
                dataset_mag=magR_box_nonan_noinf.to_dataset(name='magR')
                dataset_mag.attrs['global_attribute']= 'Stack of absolute value of ratio between temperature and salinity gradient of data for day '+str(date)
                dataset_mag.to_netcdf(path=filename,mode='w')
    
            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_'+str(date)+'.1d_'+str(box)+'.nc'
            if not os.path.exists(filename):
                
                print('compute magR for box '+str(box))
                magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
                Tu_box=Tu[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
                magR_box_nonan=magR_box[~np.isnan(magR_box)]
                Tu_box_nonan=Tu_box[~np.isnan(magR_box)]
                magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
                Tu_box_nonan_noinf=Tu_box_nonan[~np.isinf(magR_box_nonan)]
                magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
                Tu_box_nonan_noinf=Tu_box_nonan_noinf.reset_index('z')

                print('write '+filename)
                dataset_mag=Tu_box_nonan_noinf.to_dataset(name='Tu')
                dataset_mag.attrs['global_attribute']= 'Stack of turner angle of data for day '+str(date)
                dataset_mag.to_netcdf(path=filename,mode='w')


20130301
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d01.1d_gridTsurf.nc
20130302
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d02.1d_gridTsurf.nc
20130303
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d03.1d_gridTsurf.nc
20130304
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d04.1d_gridTsurf.nc
20130305
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d05.1d_gridTsurf.nc
20130306
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d06.1d_gridTsurf.nc
20130307
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d07.1d_gridTsurf.nc
20130308
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d08.1d_gridTsurf.nc
20130309
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d09.1d_gridTsurf.nc
20130310
/store/molines/NATL60/NATL60-CJM165-S/1d/2013/NATL60-CJM165_y2013m03d10.1d_gridTsurf.nc
20130311
/store/molines/NATL60

distributed.nanny - WARNING - Worker process 14865 was killed by signal 9
distributed.nanny - WARNING - Worker process 14867 was killed by signal 9
distributed.nanny - WARNING - Worker process 14869 was killed by signal 9
distributed.nanny - WARNING - Worker process 14871 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 14873 was killed by signal 9
distributed.nanny - WARNING - Worker process 14875 was killed by signal 9
distributed.nanny - WARNING - Worker process 14877 was killed by signal 9
distributed.nanny - WARNING - Worker process 14879 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KilledWorker: ('open_dataset-cc3317aaa1a2d84401c4ed72de81b98dsosaline-5f90889ed280545c565bc26c29b38a0c', <Worker 'tcp://195.83.183.130:34412', memory: 0, processing: 38>)